<a href="https://colab.research.google.com/github/kobrue02/evaluating-llm-generated-nlu-data/blob/main/bin/notebooks/data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clone the repository and cd into it

In [ ]:
!git clone https://github.com/kobrue02/evaluating-llm-generated-nlu-data/
%cd evaluating-llm-generated-nlu-data

Import the necessary classes and methods

In [ ]:
from bin.data_generation.generate_data import DataGenerationModel
from bin.data_generation.construct_prompt import load_prompt
from bin.utils.types import DataSet
from transformers import AutoModelForCausalLM, AutoTokenizer

Initialize Phi-3.5 Model

In [ ]:
phi_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
phi_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

Initialize Llama model

In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")

In [ ]:
# Load the reference data
from bin.utils.methods import load_sipgate_dataset
ref_df = load_sipgate_dataset()

Load the model

In [ ]:
phi = DataGenerationModel(model=phi_model, tokenizer=phi_tokenizer, reference_dataset=ref_df)

In [ ]:
llama = DataGenerationModel(model=model, tokenizer=tokenizer, reference_dataset=ref_df)

2025-02-04 09:58:48,714 - bin.data_generation.generate_data - INFO - Using device: cuda
INFO:bin.data_generation.generate_data:Using device: cuda
2025-02-04 09:58:48,719 - bin.data_generation.generate_data - INFO - Initializing pipeline
INFO:bin.data_generation.generate_data:Initializing pipeline


Generate data with basic prompt

In [ ]:
prompt_id = "few_shot_simple"
num_samples = 25
dataset = llama.build_dataset_from_reference(prompt_id, num_samples)
dataset.to_csv(f"{prompt_id}_data.csv", columns=["query", "intent"], encoding="utf-8")
dataset.to_data_frame()